# Лабораторная работа 6

**Внимание!** Данную лабораторную нужно выполнять на **Python 3.6+**.

## 1. Работа с метаклассами

### Задание 1.1 (0.6 балла)

Реализуйте метакласс **BoundedMeta**, который контролирует количество созданных объектов классов, которые имеют данный метакласс. Допустимое количество объектов задайте параметром (по умолчанию 1).

В случае превышения бросайте исключение **TypeError**. Eсли значение параметра - **None**, то ограничений нету.

Другими словами, у класса **C** с метаклассом **BoundedMeta** должно быть создано не более 2 экземпляров. 

In [1]:
class BoundedMeta(type):
    _counter = 0
    
    def __new__(cls, name, bases, namespace, **kargs):
        return super().__new__(cls, name, bases, namespace)

    def __init__(cls, name, bases, namespace, max_instance_count = 1, **kargs):
        cls._max_instance_count = max_instance_count
        super().__init__(name, bases, namespace)

    def __call__(cls):
        if (cls._max_instance_count is not None) and (cls._counter >= cls._max_instance_count):
            raise TypeError()
        cls._counter += 1
        return super().__call__()

class C(metaclass=BoundedMeta, max_instance_count=2):
    pass

c1 = C()
c2 = C()

try:
    c3 = C()
except TypeError:
    print('everything works fine!')
else:
    print('something goes wrong!')

everything works fine!


### Задание 1.2 (0.6 балла)

Реализуйте класс **BoundedBase**, в котором определен абстрактный classmethod get_max_instance_count, возвращающий максимальное количество экзмепляров, которые можно создать.

Не допускайте ***создания*** объекта, если данное значение превышено - бросайте исключение **TypeError**. Значение, равное **None** - без ограничений.

In [2]:
from abc import *

class BoundedBase(ABC):
    @classmethod
    @abstractmethod
    def get_max_instance_count(cls):
        pass
    
    @classmethod
    def _check_counter_var(cls):
        try:
            test = cls._counter
        except AttributeError:
            cls._counter = 0
    
    def __new__(cls, **kargs):
        cls._check_counter_var()
        max_cls_count = cls.get_max_instance_count()
        if (max_cls_count is not None) and (cls._counter >= max_cls_count):
            raise TypeError()
        cls._counter += 1
        return super().__new__(cls)
    
    
class D(BoundedBase):
    @classmethod
    def get_max_instance_count(cls):
        return 1
    
d1 = D()

try:
    d2 = D()
except TypeError:
    print('everything works fine!')
else:
    print('something goes wrong!')

everything works fine!


## 2. Работа с NumPy и SciPy

В области машинного обучения одним из самых популярных методов бинарной классификации (предсказываем один из двух классов, $+1$ или $-1$ для каждого объекта) является логистическая регрессия. Она выводится из метода максимального правдоподобия, который приводит к следующей задаче оптимизации:

$$ L(w, X, y) = \frac1{N}\sum_{i = 0}^{N} log (1 + exp(-y_ix_i^Tw)) + \frac{C}{2} ||w||^2 \longrightarrow \min_w$$
$$X \in R^{N \times M}, x \in R^{M}, w \in R^{M}, y \in \{-1, 1\}^N$$

Здесь $X$ - матрица объекты-признаки для обучающей выборки (по строкам объекты, по столбцам признаки), а $y$ - вектор ответов. Коэффициент $C$, вообще говоря, нужно подбирать отдельно, поскольку разные его значения приводят к разным решениям задачи оптимизации. Но в этой задаче положим $\mathbf{C = 10^{-3}}$

Когда мы решили задачу оптимизации (нашли $w$), мы принимаем решение о том, к какому классу относится объект по правилу $y(x) = sign(x^Tw)$.

In [3]:
import numpy as np

Для тестирования правильности вычисления сгенерируем аргументы небольшого размера

In [4]:
sample_size, features_count = 25, 10
w = np.random.random(features_count)
X, y = np.random.random((sample_size, features_count)), 2 * (np.random.randint(0, 2, sample_size) - 0.5)

In [5]:
C = 0.001

### Задание 2.1 (0.2 балла)

In [6]:
XY = np.hstack((X, y[:, np.newaxis]))
del X, y

Немного поработаем с ndarray. Получите из массива XY обратно X и y.

In [7]:
X = XY[:, :-1]
y = XY[:, -1]
print(X)
print(y)

[[ 0.47943316  0.96748275  0.21525601  0.56288243  0.34452281  0.7597561
   0.12279985  0.04224322  0.55474249  0.38358227]
 [ 0.8644011   0.18155884  0.36992103  0.6948626   0.14891005  0.77912819
   0.91893693  0.40671763  0.71905181  0.78601628]
 [ 0.5268153   0.42845457  0.46720848  0.66025677  0.80310226  0.78692024
   0.40496498  0.38841005  0.2452937   0.31195825]
 [ 0.80597293  0.22258376  0.60449312  0.99112405  0.49266365  0.46381844
   0.30652502  0.63968973  0.09166051  0.18971217]
 [ 0.08551793  0.82155052  0.58232437  0.69938756  0.18574334  0.98302113
   0.59810873  0.69226518  0.37601594  0.22270991]
 [ 0.70520798  0.11594947  0.82622207  0.86681431  0.85900211  0.432353
   0.72698221  0.70853753  0.62938548  0.31107697]
 [ 0.62550631  0.99541188  0.96827579  0.52968688  0.5370013   0.22811133
   0.80107689  0.29121422  0.80715202  0.06923274]
 [ 0.07386807  0.18628828  0.88312855  0.39229589  0.90331145  0.8492079
   0.44866773  0.90795022  0.73543276  0.4671008 ]
 [ 0

### Задание 2.2 (0.2 балла)

Запрограммируйте вычисление функции L, используйте только матричные операции (внутри не должно быть циклов).

**Замечание**: Нигде в промежуточных вычислениях не стоит вычислять значение $\exp(−y_ix^Tw)$, иначе может произойти переполнение. Вместо этого следует напрямую вычислять необходимые величины с помощью специализированных для этого функций: `np.logaddexp` для `ln(1 + exp(·))` и `sp.special.expit` для `1/(1 + exp(-(·)))`.

$$ L(w, X, y) = \frac1{N}\sum_{i = 0}^{N} log (1 + exp(-y_ix_i^Tw)) + \frac{C}{2} ||w||^2$$

In [8]:
import numpy as np
import numpy.linalg as LA

def logistic(w, X, y):
    """
        logistic(w, X, y) вычисляет функцию качества лог регрессии L(w, X, y)
        
        w: np.array размера (M,)
        X: np.array размера (N, M)
        y: np.array размера (M,)
        
        funcw: np.float 
    """
    return np.sum(np.logaddexp(0, -1 * y * np.dot(X, w))) / len(X) + C * (LA.norm(w) ** 2) / 2.0

### Задание 2.3.1 (0.3 балла)

Найдите градиент функции $\nabla_w L(w, X, y)$, запишите в терминах матричных операций.

Эффективно запрограммируйте вычисление градиента (опять же, только матричные операции!)

$$\nabla_{w_k} L(w, X, y) = \frac1{N}\sum_{i = 0}^{N} -\frac{y_ix_{i,k}}{e^{y_ix_{i}^Tw}+1}+C ||w_k||$$

In [9]:
from scipy.special import expit
import numpy.linalg as LA

def logistic_grad(w, X, y):
    '''
        logistic_grad(w, X, y) вычисляет градиент функции качества лог регрессии dL(w, X, y)/dw
        
        w: np.array размера (M,)
        X: np.array размера (N, M)
        y: np.array размера (M,)
        
        gradw: np.array размера (M,)
    '''
    def grad(k):
        return np.sum((-1 * y * X.T[k]) * expit(-1 * y * np.dot(X, w))) / len(X) + C * LA.norm(w[k])
    return np.fromfunction(np.vectorize(grad), w.shape, dtype=int)

In [10]:
assert(logistic_grad(w, X, y).shape == w.shape)

### Задание 2.3.2 (0.3 балла)

Очень часто при подсчёте градиента допускаются ошибки, проверьте правильность реализации подсчёта градиента с помощью конечных разностей. 

$$[\nabla f(x)]_i \approx \frac{f(x + \varepsilon \cdot e_i) - f(x)}{\varepsilon}~~~~$$

где $e_i = (0, ... , 0, 1, 0, ..., 0)$ - i-й базисный орт, $\epsilon \approx 10^{-8}$

In [11]:
def max_error(a, b): 
    return np.max(np.abs(a - b))


def grad_finite_diff(func, x, eps=1e-8):
    """
        w: np.array размера (M,)
        func: скалярная функция от векторного аргумента w, func(w) =  число
        eps: np.float константа для проверки градиента
        
        dnum: np.array размера (M,), численно посчитанный градиент
    """
    x, fval, dnum = x.astype(np.float64), func(x), np.zeros_like(x)

    def grad_diff(i):
        e_i = np.zeros_like(w); e_i[i] = 1
        return (func(x + eps * e_i) - fval) / eps
    
    return np.fromfunction(np.vectorize(grad_diff), w.shape, dtype=int)

In [12]:
mat_grad = logistic_grad(w, X, y)
num_grad = grad_finite_diff(lambda w: logistic(w, X, y), w)

err = max_error(mat_grad, num_grad)
print('err = ', err, 'ok' if err < 1e-6 else 'ошибка очень большая!')

err =  3.26100205728e-08 ok


### Задание 2.4.1 (0.4 балла)

Для некоторых задач оптимизации очень удобно использовать гессиан.

Вычислите гессиан для функции L, запишите ответ в терминах матричных операций. Эффективно запрограммируйте вычисление гессиана.

$$\nabla_{w_iw_j} L(w, X, y) = \frac1{N}\sum_{k = 0}^{N} \frac{y_k^2x_{k,i}x_{k,j}e^{y_kx_{k}^Tw}}{(e^{y_kx_{k}^Tw}+1)^2}+C_{[i == j]}$$

In [13]:
def logistic_hess(w, X, y):
    """
        logistic_hess(w, X, y) вычисляет гессиан функции качества лог регрессии dL(w, X, y)/dw
        
        w: np.array размера (M,)
        X: np.array размера (N, M)
        y: np.array размера (M,)
        
        hessw: np.array размера (M, M)
    """
    
    def gaussian(i, j):
        part_sums = ((y ** 2) * X.T[i] * X.T[j]) * expit(-1 * y * np.dot(X, w)) * expit(y * np.dot(X, w))
        return np.sum(part_sums) / len(X) + C * (i == j)

    return np.fromfunction(np.vectorize(gaussian), (w.shape[0], w.shape[0]), dtype=int)

In [14]:
assert(logistic_hess(w, X, y).shape == (w.shape[0], w.shape[0]))

### Задание 2.4.2 (0.4 балла)

Теперь проверим правильность реализации подсчёта гессиана.

Для гессиана проверка выглядит похожим образом

$$[\nabla^2 f(x)]_{ij} \approx \frac{f(x + \varepsilon \cdot e_i + \varepsilon \cdot e_j) -f(x + \varepsilon \cdot e_i) - f(x + \varepsilon \cdot e_j)+ f(x)}{\varepsilon^2}~~~~~~~~~~~~~~~~~~~~~$$

где $e_i = (0, ... , 0, 1, 0, ..., 0)$ - i-й базисный орт, $\varepsilon \approx 10^{-4}$

In [15]:
def hess_finite_diff(func, w, eps=1e-4):
    '''
        w: np.array размера (M,)
        func: скалярная функция от векторного аргумента w, func(w) =  число
        eps: np.float константа для проверки градиента
        
        dnum: np.array размера (M,), численно посчитанный градиент
    '''
    w, fval, dnum = w.astype(np.float64), func(w).astype(np.float64), np.zeros((w.size, w.size), dtype=np.float64)

    def hess_diff(i, j):
        e_i = np.zeros_like(w); e_i[i] = 1
        e_j = np.zeros_like(w); e_j[j] = 1
        return (func(w + eps * (e_i + e_j)) - func(w + eps * e_i) - func(w + eps * e_j) + fval) / (eps ** 2)
    
    return np.fromfunction(np.vectorize(hess_diff), (w.shape[0], w.shape[0]), dtype=int)

In [16]:
mat_grad = logistic_hess(w, X, y)
num_grad = hess_finite_diff(lambda w: logistic(w, X, y), w)

err = max_error(mat_grad, num_grad)

print('err = ', err)
print('ok' if max_error(mat_grad, num_grad) < 1e-4 else 'ошибка очень большая!')

err =  1.68779529248e-06
ok
